<a href="https://colab.research.google.com/github/JamesLabUofT/GEE_Workshop/blob/main/scripts/working_with_awesome_gee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extracting and Processing data from Awesome GEE Community Catalog


## Module Overview

This module introduces participants to accessing, analyzing, and visualizing forest-related geospatial datasets from the Awesome GEE Community Data Catalog using Google Earth Engine (GEE) and Python. Through hands-on exercises in Google Colab, participants will explore forest disturbance, land cover, and species composition across Canada using high-resolution datasets.

## Learning Objectives:
By the end of this module, participants will be able to:

* Understand the structure and content of selected forest datasets in the Awesome GEE Catalog
* Use Python and the Earth Engine API to access, filter, and visualize geospatial data
* Work with both raster (ImageCollections) and vector (FeatureCollections) datasets
* Perform spatial and temporal analysis on forest disturbance and land cover
Export processed data for further analysis or reporting

## Topics Covered:
**Introduction to the Awesome GEE Community Catalog**

* Overview of the catalog and its purpose
* Navigating the catalog and locating datasets

**Working with FeatureCollections: NBAC Dataset
Dataset: National Burned Area Composite (NBAC)**

* Understanding vector data in GEE
* Loading and filtering NBAC by year and region
* Visualizing burned areas on interactive maps
* Calculating burned area statistics
* Exporting filtered data to CSV or Google Drive

**Using the NBAC perimeters, lets explore some forest cover**

**Exploring Raster Datasets**

a. High-Resolution Annual Forest Land Cover (1984–2022)
* Visualizing annual forest cover
* Detecting forest cover change over time
* Creating animations or time sliders

b. Canada Landsat-Derived Forest Harvest Disturbance (1985–2020)
* Mapping harvest disturbances
* Comparing harvest vs. fire impacts
* Temporal analysis of disturbance trends

c. Canada Long-Term Tree Species (1984–2022)
* Mapping dominant tree species
* Analyzing species distribution changes
* Overlaying with land cover for ecological insights

**Data Extraction and Visualization**

* Reducing data over custom regions (e.g., provinces, ecozones)
* Exporting time-series data to Pandas DataFrames
* Creating charts and maps using matplotlib and geemap
* Exporting results for use in reports or further analysis



## Introduction to the Awesome GEE Community Catalog

## Introduction to the Awesome GEE Community Catalog


✅ Expand access to valuable datasets not included in the official GEE catalog
🌍 Support reproducible science by sharing preprocessed, ready-to-use data
🧑‍🤝‍🧑 Foster collaboration across the global geospatial community
📈 Accelerate research in climate, ecology, forestry, urban studies, and more

https://gee-community-catalog.org/

**environment setup**

In [ ]:
import ee
import geemap
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ipyleaflet

Authenticate and intialize ee

In [ ]:
ee.Initialize() # project='ee-userid'

# Working with Feature Collections: NBAC

In this section, participants will explore how to work with vector data in Google Earth Engine using the NBAC dataset, which maps burned areas across Canada. This hands-on module introduces the concept of FeatureCollections, a core data structure in GEE used to represent geographic features like points, lines, and polygons.

FeatureCollections are the vector equivalent of ImageCollections in Earth Engine. They store geographic features—such as fire perimeters, forest plots, or administrative boundaries—along with associated attributes (e.g., year, area, cause). In this session, we’ll use the National Burned Area Composite (NBAC) to:

* Load and explore a FeatureCollection
* Filter features by year and region
* Visualize fire perimeters on an interactive map
* Calculate statistics like total burned area
* Export selected features for further analysis

We'll start with loading in the NBAC from https://gee-community-catalog.org/projects/nbac/#citation

In [ ]:
nbac = ee.FeatureCollection("projects/sat-io/open-datasets/CA_FOREST/NBAC/nbac_1972_2023_20240530")


Lets look at the data

In [ ]:
print(nbac.size().getInfo(), nbac.first().propertyNames().getInfo())

Lets get all fires from 2023

In [ ]:
nbac_2023 = nbac.filter(ee.Filter.eq('YEAR', 2023))
print(nbac_2023.size().getInfo())

Let's look just at the fires in Ontario

https://cwfis.cfs.nrcan.gc.ca/downloads/nbac/NBAC_1972to2024_20250506_shp_metadata.pdf

In [ ]:
nbac_on_2023 = nbac_2023.filter(ee.Filter.eq('ADMIN_AREA', 'ON'))
nbac_on_2023.size().getInfo()

lets visualize it

In [ ]:
centroid = nbac_on_2023.geometry().centroid(1).getInfo()['coordinates']
# geemap often automatically zooms to the layer extent when added,
# but explicitly setting a zoom level can help if needed.
# A zoom level between 8 and 10 is usually good for a provincial park.
zoom_level = 9

m = geemap.Map(center=(centroid[1], centroid[0]), zoom=zoom_level)

# Visualization parameters are not strictly needed for feature collections/geometries,
# but we can add a simple outline color if the default is hard to see.
fires_vis = {'color': 'FF0000'} # Red outline

# Add the AOI layer using aoi_ee
m.add_layer(nbac_on_2023, fires_vis, "Wildfires 2023")

# Add the layer manager to easily toggle layers
m.add_layer_manager()

m

Lets calculate some summary statistics

total area burned?

In [ ]:
# Calculate the area of each feature and add it as a property
def add_area(feature):
  return feature.set({'area_sq_ha': feature.geometry().area().divide(10000)})

nbac_on_2023 = nbac_on_2023.map(add_area)

# Calculate the total area
total_area = nbac_on_2023.aggregate_sum('area_sq_ha')

# Print the total area
print('Total area:', total_area.getInfo())


## Exploring raster datasets using NBAC feature collection


[High-Resolution Annual Forest Land Cover (1984–2022)](https://gee-community-catalog.org/projects/ca_lc/)

In [ ]:
forest_lc = ee.ImageCollection("projects/sat-io/open-datasets/CA_FOREST_LC_VLCE2")
forest_lc